In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle5 as pickle

In [4]:
# HIDE OUTPUT
#!wget http://www.atarimania.com/roms/Roms.rar 
#!unrar x -o+ ./Roms.rar >./roms
!python3 -m atari_py.import_roms ./ROMS >./roms

In [2]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/MsPacman-v5")

Action Space: Discrete(9)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [3]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 7
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 3
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 2
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [8]:
n_actions = 18
embed_size = 62#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open(root_dir+f'./genoms/best_genom_pacman.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

In [6]:
def test_atary(genom, draw=False,game_name="ALE/MsPacman-v5",seed=0):

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  np.random.seed(seed)
  env.seed(seed)
  for i in range(1600):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
      t=pd.Timestamp.now()
      #state = encode_mobnet(np.array([state]))
      state = encode_simple(state)
      reward_sum += reward
      if np.shape(state)[0]>1:
          shp = np.shape(state)
          state = np.reshape(state,[1,shp[0]])
      out_tape = controller.act(state,reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum

In [5]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/MsPacman-v5"
  #один энвайронмент, разные сиды
  q_arr = []
  for seed in range(3):
    q_arr.append(test_atary(genom, draw=draw,game_name=game_name,seed=seed))
  return np.mean(q_arr)-0.000000001*np.sum(genom**2)

In [9]:
%%time
q=test_atary(genom,game_name="ALE/MsPacman-v5", draw=False)
print('q=',q)

q= 70.0
CPU times: user 17.7 s, sys: 38.5 s, total: 56.1 s
Wall time: 1min 8s


In [10]:
%%time
q=multy_test(genom)
print('q=',q)
#q=multy_test(genom)
#print('q=',q)

In [ ]:
%%time
q=test_atary(genom)
print('q=',q)

In [ ]:
print(pd.Timestamp.now())

In [ ]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=12)

In [ ]:
try:
    with open('genoms/best_genom_pacman.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

In [ ]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_pacman.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
scores for optimizers augmented [3.03233333e+02 1.00000000e+10 1.00000000e+10 1.00000000e+10
 1.00000000e+10 1.00000000e+10 1.00000000e+10 1.00000000e+10]
chosen evol_mid_chaos previous_result: nan per tacts: nan
iteration 0 y= [1163.33331044  793.33331045  613.33331044]
iteration 1 y= [1163.33331044  793.33331045  613.33331044]
iteration final y= [1163.33331044  793.33331045  613.33331044]
result evol_mid_chaos previous_gain: nan per tacts: nan duration 0 days 00:32:38.487090
WRITTEN
opt# 1
random trial
scores for optimizers augmented [3.03732276e+02 1.89996297e+00 1.00000000e+10 1.00000000e+10
 1.00000000e+10 1.00000000e+10 1.00000000e+10 1.00000000e+10]
chosen gradient_wide_50 previous_result: nan per tacts: nan
score_new 793.3333104443182 score_prev 1163.3333104445442 gained -370.000000000226
undo
score_new 793.3333104443182 score_prev 1163.3333104445442 gained -370.000000000226
undo
score_new 793.3333104443182 score_prev 1163.3333104445442 gained -370.000000000226
undo
scor

KeyboardInterrupt: 

In [19]:
with open('genoms/best_genom_pacman.pkl', 'rb') as f:
    genom = pickle.load(f)[-1]
q=multy_test(genom,draw=True)
print('q=',q)

q= 1163.3333104451385


In [20]:
video[0].save(
    './out_videos/seaquest_pacman.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)